In [1]:
# William Nourse
# Feb 15 2021

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib import rc
from ipywidgets import interact_manual
%matplotlib widget

In [2]:
Cm = 50 # nF
Gm = 1 # uS
Er = -60 # mV
R = 20 # mV

# Design the synapse
Einh = -65
delE = Einh-Er
g=-R/delE

In [3]:
# Plot the equilibrium manifold and phase space
U1 = np.linspace(0,R)
U2 = delE*(R-U1)/(delE-U1)

def slopefie(U1,U2):
    dU1 = 1/Cm*(-U1+g*U2/R*(delE-U1)+R)
    dU2 = 1/Cm*(-U2+g*U1/R*(delE-U2)+R)
    return dU1,dU2

U1ax,U2ax=np.meshgrid(U1,U1)
dU1,dU2 = slopefie(U1ax,U2ax)

plt.figure(figsize=(15,15))
plt.plot(U1,U2,label='Equilibrium Manifold',linewidth=10)
plt.plot(U1,U1,linestyle=':',label='$U_1^*=U_2^*$',linewidth=5)
plt.quiver(U1ax,U2ax,dU1,dU2,scale=50)
plt.xlabel('$U_1$')
plt.ylabel('$U_2$')
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [4]:
dt = 1 # ms
tStart = 200 # ms
tEnd = 700 # ms
tMax = 1000 # ms
t = np.arange(0,tMax,dt)
numSteps = np.size(t)
tPred = np.arange(tStart,tEnd,dt)

I = 1 # nA
Iapp = np.zeros(numSteps)
for i in range(numSteps):
    if (t[i]>tStart) and (t[i]<tEnd):
        Iapp[i]=I
        
# Calculate k_i
kiLL = 1/(Cm*(2+g))
kiUL = (1+g)/(Cm*(2+g))
kiMean = 1/(2*Cm)

# Compute U(t)
U1sim = np.zeros(numSteps)
U2sim = np.zeros(numSteps)

for i in range(1,numSteps):
    U1sim[i] = U1sim[i-1] + dt/Cm*(-Gm*U1sim[i-1] + min(max(U2sim[i-1]/R,0),1)*g*(delE-U1sim[i-1])+Iapp[i-1]+R)
    U2sim[i] = U2sim[i-1] + dt/Cm*(-Gm*U2sim[i-1] + min(max(U1sim[i-1]/R,0),1)*g*(delE-U2sim[i-1])+R)

In [5]:
plt.figure(figsize=(30,15))
plt.subplot(2,2,1)
plt.plot(t,Iapp,lw=10)
plt.xlabel('Time (ms)')
plt.ylabel('$I_{app} (nA)$')

plt.subplot(2,2,3)
plt.plot(t,U1sim,lw=10,label='$U_1$')
plt.plot(t,U2sim,lw=10,label='$U_2$',linestyle='--')
UpredLL = U1sim[round(tStart/dt)]+kiLL*I*(tPred-tStart)
UpredUL = U1sim[round(tStart/dt)]+kiUL*I*(tPred-tStart)
UpredMean = U1sim[round(tStart/dt)]+kiMean*I*(tPred-tStart)
plt.plot(tPred,UpredLL,label='Low Pred',lw=5)
plt.plot(tPred,UpredUL,label='Upper Pred',lw=5)
plt.plot(tPred,UpredMean,label='Mean Pred',lw=5)
plt.xlabel(('Time (ms)'))
plt.ylabel(('U (mV)'))
plt.legend()

plt.subplot(2,2,(2,4))
plt.plot(U1,U2,label='Equilibrium Manifold',linewidth=5)
plt.plot(U1,U1,linestyle=':',label='$U_1^*=U_2^*$',linewidth=5)
plt.plot(U1sim,U2sim,label='State',linewidth=5)
plt.xlabel('$U_1$')
plt.ylabel('$U_2$')
plt.legend()

plt.savefig('Integration_Demo.png')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
# Pulsed positive current input

tOn = [200,400,600,800] # ms
tOff = [300,500,700,900] # ms
tMax = 1000 # ms
t = np.arange(0,tMax,dt)
numSteps = np.size(t)
tPred = np.arange(tStart,tEnd,dt)

I = 1 # nA
Iapp = np.zeros(numSteps)
for index in range(len(tOn)):
    for i in range(numSteps):
        if (t[i]>tOn[index]) and (t[i]<tOff[index]):
            Iapp[i]=I
        
# Calculate k_i
kiLL = 1/(Cm*(2+g))
kiUL = (1+g)/(Cm*(2+g))
kiMean = 1/(2*Cm)

# Compute U(t)
U1sim = np.zeros(numSteps)
U2sim = np.zeros(numSteps)

for i in range(1,numSteps):
    U1sim[i] = U1sim[i-1] + dt/Cm*(-Gm*U1sim[i-1] + min(max(U2sim[i-1]/R,0),1)*g*(delE-U1sim[i-1])+Iapp[i-1]+R)
    U2sim[i] = U2sim[i-1] + dt/Cm*(-Gm*U2sim[i-1] + min(max(U1sim[i-1]/R,0),1)*g*(delE-U2sim[i-1])+R)
    
plt.figure(figsize=(30,15))
plt.subplot(2,1,1)
plt.plot(t,Iapp,lw=10)
plt.xlabel('Time (ms)')
plt.ylabel('$I_{app} (nA)$')

plt.subplot(2,1,2)
plt.plot(t,U1sim,lw=10,label='$U_1$')
plt.plot(t,U2sim,lw=10,label='$U_2$',linestyle='--')
plt.vlines(tOn,ymin=0,ymax=max(U1sim)+1,color='k',linestyle=':')
plt.vlines(tOff,ymin=0,ymax=max(U1sim)+1,color='k',linestyle=':')
plt.xlabel(('Time (ms)'))
plt.ylabel(('U (mV)'))
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
# Alternating Current input

tStart = 200
tOn = [200,400,600,800] # ms
tOff = [300,500,700,900] # ms
tMax = 1000 # ms
t = np.arange(0,tMax,dt)
numSteps = np.size(t)
tPred = np.arange(tStart,tEnd,dt)

I = 1 # nA
Iapp = np.zeros(numSteps)
for i in range(numSteps):
    if t[i]>tStart:
        Iapp[i] = -I
for index in range(len(tOn)):
    for i in range(numSteps):
        if (t[i]>tOn[index]) and (t[i]<tOff[index]):
            Iapp[i]=I

        
# Calculate k_i
kiLL = 1/(Cm*(2+g))
kiUL = (1+g)/(Cm*(2+g))
kiMean = 1/(2*Cm)

# Compute U(t)
U1sim = np.zeros(numSteps)
U2sim = np.zeros(numSteps)

for i in range(1,numSteps):
    U1sim[i] = U1sim[i-1] + dt/Cm*(-Gm*U1sim[i-1] + min(max(U2sim[i-1]/R,0),1)*g*(delE-U1sim[i-1])+Iapp[i-1]+R)
    U2sim[i] = U2sim[i-1] + dt/Cm*(-Gm*U2sim[i-1] + min(max(U1sim[i-1]/R,0),1)*g*(delE-U2sim[i-1])+R)
    
plt.figure(figsize=(30,15))
plt.subplot(2,1,1)
plt.plot(t,Iapp,lw=10)
plt.xlabel('Time (ms)')
plt.ylabel('$I_{app} (nA)$')

plt.subplot(2,1,2)
plt.plot(t,U1sim,lw=10,label='$U_1$')
plt.plot(t,U2sim,lw=10,label='$U_2$',linestyle='--')
plt.vlines(tOn,ymin=0,ymax=max(U1sim)+1,color='k',linestyle=':')
plt.vlines(tOff,ymin=0,ymax=max(U1sim)+1,color='k',linestyle=':')
plt.xlabel(('Time (ms)'))
plt.ylabel(('U (mV)'))
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
"""
# Code for generating the animation, takes too long to run in class but could be nice at home

fig = plt.figure(figsize=(30,15))
axIapp = plt.subplot(2,2,1)
axU = plt.subplot(2,2,3)
axPhase = plt.subplot(2,2,(2,4))

axIapp.set_xlim((0,tMax))
axIapp.set_ylim((0,I))
axIapp.set_xlabel('Time (ms)')
axIapp.set_ylabel('$I_{app}$')

axU.set_xlim((0,tMax))
axU.set_ylim((0,R))
axU.set_xlabel(('Time (ms)'))
axU.set_ylabel(('U (mV)'))

axPhase.set_xlim((0,R))
axPhase.set_ylim((0,R))
axPhase.set_xlabel('$U_1$')
axPhase.set_ylabel('$U_2$')

current, = axIapp.plot([],[],lw=10)

U1curve, = axU.plot([],[],lw=10,label='$U_1$')
U2curve, = axU.plot([],[],lw=10,label='$U_2$',linestyle='--')
UpredLL = U1sim[round(tStart/dt)]+kiLL*I*(tPred-tStart)
UpredUL = U1sim[round(tStart/dt)]+kiUL*I*(tPred-tStart)
UpredMean = U1sim[round(tStart/dt)]+kiMean*I*(tPred-tStart)
axU.plot(tPred,UpredLL,label='Low Pred',lw=5)
axU.plot(tPred,UpredUL,label='Upper Pred',lw=5)
axU.plot(tPred,UpredMean,label='Mean Pred',lw=5)
axU.legend()

axPhase.plot(U1,U2,label='Equilibrium Manifold',linewidth=5)
axPhase.plot(U1,U1,linestyle=':',label='$U_1^*=U_2^*$',linewidth=5)
U1Phase, = axPhase.plot([],[],lw=5,label='$U_1$')
axPhase.legend()

def drawframe(i):
    #print(i)
    current.set_data(t[:i],Iapp[:i])
    U1curve.set_data(t[:i],U1sim[:i])
    U2curve.set_data(t[:i],U2sim[:i])
    U1Phase.set_data(U1sim[:i],U2sim[:i])
    
    return(current,U1curve,U2curve,U1Phase)
    


anim = animation.FuncAnimation(fig, drawframe, frames=numSteps,interval=0.5, blit=True,repeat=False,save_count=numSteps)
"""

"\n# Code for generating the animation, takes too long to run in class but could be nice at home\n\nfig = plt.figure(figsize=(30,15))\naxIapp = plt.subplot(2,2,1)\naxU = plt.subplot(2,2,3)\naxPhase = plt.subplot(2,2,(2,4))\n\naxIapp.set_xlim((0,tMax))\naxIapp.set_ylim((0,I))\naxIapp.set_xlabel('Time (ms)')\naxIapp.set_ylabel('$I_{app}$')\n\naxU.set_xlim((0,tMax))\naxU.set_ylim((0,R))\naxU.set_xlabel(('Time (ms)'))\naxU.set_ylabel(('U (mV)'))\n\naxPhase.set_xlim((0,R))\naxPhase.set_ylim((0,R))\naxPhase.set_xlabel('$U_1$')\naxPhase.set_ylabel('$U_2$')\n\ncurrent, = axIapp.plot([],[],lw=10)\n\nU1curve, = axU.plot([],[],lw=10,label='$U_1$')\nU2curve, = axU.plot([],[],lw=10,label='$U_2$',linestyle='--')\nUpredLL = U1sim[round(tStart/dt)]+kiLL*I*(tPred-tStart)\nUpredUL = U1sim[round(tStart/dt)]+kiUL*I*(tPred-tStart)\nUpredMean = U1sim[round(tStart/dt)]+kiMean*I*(tPred-tStart)\naxU.plot(tPred,UpredLL,label='Low Pred',lw=5)\naxU.plot(tPred,UpredUL,label='Upper Pred',lw=5)\naxU.plot(tPred,Upred